In [4]:
import os 
from numba import njit
import malariagen_data
import zarr
import allel
import pandas as pd
import numpy as np
import polars as pl
from tqdm.notebook import tqdm

from bokeh.io import output_notebook # enables plot interface in J notebook
output_notebook(hide_banner=True)

### IBD segment detection with unphased data

The method is loosely based on the following studies:   
https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0034267  
https://www.sciencedirect.com/science/article/pii/S0002929720300549  

Essentially, we take two individuals and scan across their genomes. We aim to identify segments in so-called half-IBD (IBD1), where one chromosome is shared, and full-IBD or IBD2, where both chromosomes are shared. 

To detect IBD1 segment breakpoints, we record the position of any SNP where the two individuals are homozygous but different alleles - this has to be an IBD1 breakpoint.

To detect IBD2 segment breakpoints, we record the position of any SNP where alleles are different, i.e we are looking for stretches where everything is identical.

We then apply a size filter to remove small segments which are identical by state but probably not identical by descent. At the minute, this is somewhat arbitrary, and we need to develop sensible thresholds or decide if we want to try and do clever things to merge broken fragments. Although sequencing errors and mutations will currently break up IBD segments, for sibship inference at higher degrees of relatedness this will probably be OK.

In [5]:
def load_genotypes(self, contig, sample_sets, sample_query, site_mask='gamb_colu'):
    """
    Loads SNP calls and throws out invariant, singleton and doubleton sites, which
    massively speeds up IBD scanning algorithm.
    """
    
    print("Loading genotypes")
    ds_snps = self.snp_calls(
        region=contig, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        site_mask=site_mask
    )
    
    geno = allel.GenotypeDaskArray(ds_snps['call_genotype'].values)
    
    sample_ids = ds_snps['sample_id'].values
    print("computing allele counts")
    ac = geno.count_alleles().compute()

    seg = ac.is_segregating()
    dbl = ac.is_doubleton(allele=1)
    dbl2 = ac.is_doubleton(allele=2)
    dbl3 = ac.is_doubleton(allele=3)
    dbl = np.logical_or(dbl, np.logical_or(dbl2, dbl3))
    sngl = ac.is_singleton(allele=1)
    sngl2 = ac.is_singleton(allele=2)
    sngl3 = ac.is_singleton(allele=3)
    
    mask = np.logical_and(seg, ~np.logical_or(dbl, np.logical_or(sngl, np.logical_or(sngl2, sngl3))))
    print(f"retaining {mask.sum()} variants and removing {(~mask).sum()}")
    
    geno = geno[mask].compute().values
    pos = ds_snps['variant_position'].values[mask]
    
    return geno, pos, sample_ids


@njit()
def ibdscans(geno, pos):
    """
    Scans across two genotype vectors, checking and recording SNPs which are either 
    are homozygous but different (IBD1 breakpoint) or have any alleles different (IBD2 breakpoint)
    """

    geno1 = geno[:, 0]
    geno2 = geno[:, 1]

    ibd1_breakpoints = []
    #ibd2_breakpoints = []
    # Scan right along genome, as long as two inds are not both homozygous but different
    for pos_idx in range(pos.shape[0]):
        
        gn1 = geno1[pos_idx]
        gn2 = geno2[pos_idx]        
        
#         # if both alleles are not identical, record the position
#         if not (gn1 == gn2).all() and (-1 not in gn1 and -1 not in gn2):
#             ibd2_breakpoints.append(pos_idx)
            # if both alleles are not identical and not missing, and both are homozygous, record the position
        if (gn1[0] == gn1[1]) and (gn2[0] == gn2[1]) and ((gn1 != gn2).all()) and (-1 not in gn1 and -1 not in gn2):
            ibd1_breakpoints.append(pos_idx)
   
    return np.array(ibd1_breakpoints)#, np.array(ibd2_breakpoints)

def detect_ibd_segments(self, contig, sample_sets, sample_query, cohort_id, out_dir, site_mask='gamb_colu', min_size=5000):
    """
    Loads genotype data, and runs IBD scan on each pair of individuals, writing IBD segments to 
    a .tsv file.
    """
    from itertools import combinations
    
    geno, pos, sample_ids = load_genotypes(
        self=self,
        contig=contig, 
        sample_sets=sample_sets, 
        sample_query=sample_query, 
        site_mask=site_mask
    )
    
    print(f"running {cohort_id} - {geno.shape[1]} samples")
    
    zarr.save(f"{out_dir}{cohort_id}/breakpoints/{contig}/POS", pos)
    zarr.save(f"{out_dir}{cohort_id}/sample_ids.zarr", sample_ids)
    for x,y in tqdm(combinations(range(geno.shape[1]), 2)):
        ibd1 = ibdscans(geno.take([x,y], axis=1), pos)
        zarr.save(f"{out_dir}{cohort_id}/breakpoints/{contig}/{x}-{y}", ibd1=ibd1)
        

In [6]:
def load_ibd_zarr_to_dataframe(self, contigs, out_dir, cohort_id):
    from itertools import combinations
    dfs = []
    
    sample_ids = zarr.load(out_dir + f"{cohort_id}/sample_ids.zarr")
    for contig in contigs:
        pos = zarr.load(out_dir + f"{cohort_id}/breakpoints/{contig}/POS/")
        
        for x,y in tqdm(combinations(range(len(sample_ids)), 2)):
            arr = zarr.load(out_dir + f"{cohort_id}/breakpoints/{contig}/{x}-{y}/")
            
            df1 = breakpoints_to_polars_df(array=arr['ibd1'], contig=contig, pos=pos, x=x, y=y, sample_ids=sample_ids)
            dfs.append(df1)

    return pl.concat(dfs)
        
def breakpoints_to_polars_df(array, pos, contig, x, y, sample_ids):
    
    pos_ibd = pos[array]
    n_snps = np.ediff1d(array)
    sizes = np.ediff1d(pos_ibd)
    df = pl.DataFrame({'size':sizes,
                      'n_snps':n_snps,
                      'start':pos_ibd[:-1], 
                      'end':pos_ibd[1:]})
    
    df = df.with_columns(pl.lit(contig).alias("contig"),
                         pl.lit(x).alias("idx1"),
                         pl.lit(y).alias("idx2"),
                         pl.lit(sample_ids[x]).alias("sample_id1"),
                         pl.lit(sample_ids[y]).alias("sample_id2"))

    return df

def fraction_genome_ibd(self, ibd_df, cohort_id=None, min_ibd1_snps=10_000, out_dir=None, contigs=('2RL', '3RL', 'X')):
    import polars as pl
    genome_size = np.sum([self.genome_sequence(contig).shape[0] for contig in contigs])
    
    ibd_df = ibd_df.filter(pl.col('n_snps') > min_ibd1_snps)
    
    df = ibd_df.groupby(['idx1', 'idx2']).agg(pl.sum("size")).with_columns(
        (pl.col("size") / genome_size).alias("ibd1_fraction")).sort(['idx1', 'idx2'])
    
    if out_dir:
        df.write_csv(out_dir + cohort_id + "/ibd1_fraction.tsv", separator="\t")
    
    return df 

In [7]:
def plot_ibd_segments_contig(
    self,
    df,
    contig, 
    filter_expr=pl.col('n_snps') > 10_000,
    show=False,
    width=None,
    height=400
):
    from itertools import combinations
    import bokeh.plotting as bkplt
    import bokeh.models as bkmod
    import bokeh.layouts as bklay
    
    df = df.filter(pl.col('contig') == contig).filter(filter_expr)
        
    print("finding levels")
    max_inds = df['idx1'].max() + 1
    level_df = pl.DataFrame(list(combinations(range(max_inds), 2))).transpose()
    level_df = level_df.with_columns(pl.lit(np.linspace(0, 1, len(list(combinations(range(max_inds), 2))))).alias("level"))
    level_df = level_df.rename({'column_0':'idx1', 'column_1':'idx2'}).with_columns(pl.col("idx1").cast(pl.Int32),
                                                                                   pl.col("idx2").cast(pl.Int32))
    df = df.join(level_df, on=['idx1', 'idx2'])
    
#     print("colour mapping")
#     colour_mapping = {'half-ibd':'gray', 
#                       'full-ibd':'blue'}
#     colour = df['ibd_type'].apply(lambda x: colour_mapping[x])

    source = bkmod.ColumnDataSource(data={
        'index1': df['idx1'].to_numpy(),
        'index2': df['idx2'].to_numpy(),
         'sample_id1':df['sample_id1'].to_numpy(),
         'sample_id2':df['sample_id2'].to_numpy(),
        'chromosome': df['contig'].to_numpy(),
        'start': df['start'].to_numpy(),
        'end': df['end'].to_numpy(),
        'bottoms':df['level'].to_numpy(),
        'tops': df['level'].to_numpy()+0.0001,
#         'colour': colour.to_numpy()
    })

    hover = bkmod.HoverTool(tooltips=[
            ("index1", '@index1'),
            ("index2", '@index2'),
             ("sample_id1", '@sample_id1'),
             ("sample_id2", '@sample_id2'),
            ("segment span", "@start{,} - @end{,}"),
        ])
        
    print("making figure")
    if not width:
        width = int(self.genome_sequence(contig).shape[0]/200000)
    fig1 = bkplt.figure(title=contig,
                        width=width,
                        height=500, 
                        tools="tap,box_zoom,xpan,xzoom_in,xzoom_out,xwheel_zoom,reset".split() + [hover],
                        toolbar_location='above', active_drag='xpan', active_scroll='xwheel_zoom',
                        output_backend="webgl")

    glyph = bkmod.Quad(left='start', right='end', bottom='bottoms', top='tops', line_color="grey", line_alpha=.8, line_width=1)
    fig1.add_glyph(source, glyph)

    fig1.x_range = bkmod.Range1d(0, self.genome_sequence(contig).shape[0], bounds='auto')
    fig1.y_range = bkmod.Range1d(0, 1, bounds='auto')
    fig1.x_range.max_interval = self.genome_sequence(contig).shape[0]
    fig1.yaxis.visible = False
    fig1.ygrid.visible = False
    _bokeh_style_genome_xaxis(fig1, contig)
    
    if show:
        bkplt.show(fig1)
    
    return fig1

def plot_ibd_segments(
        self,
        df, 
        out_dir,
        cohort_id,
        contigs=('2RL', '3RL', 'X'),
        filter_expr=pl.col('n_snps') > 10_000,
        show=True,
        title=None,
    ):
    import bokeh.models as bkmod
    import bokeh.layouts as bklay
    import bokeh.plotting as bkplt
    
    figs = [
            plot_ibd_segments_contig(
                self=self,
                df=df,
                contig=contig,
                filter_expr=filter_expr,
                ) 
            for contig in tqdm(contigs)
            ]
    
    if out_dir:
        bkplt.output_file(filename=out_dir + cohort_id + "_segments.html", title=title)

    fig = bklay.gridplot(
        figs,
        ncols=len(contigs),
        toolbar_location="above",
        merge_tools=True,
    ) 
    
    if out_dir:
        bkplt.save(fig)
    
    if show:
        bkplt.show(fig)
    
    return fig
    
def _bokeh_style_genome_xaxis(fig, contig):
    import bokeh.models as bkmod
    """Standard styling for X axis of genome plots."""
    fig.xaxis.axis_label = f"Contig {contig} position (bp)"
    fig.xaxis.ticker = bkmod.AdaptiveTicker(min_interval=1)
    fig.xaxis.minor_tick_line_color = None
    fig.xaxis[0].formatter = bkmod.NumeralTickFormatter(format="0,0")

In [8]:
#https://raw.githubusercontent.com/anopheles-genomic-surveillance/selection-atlas/main/workflow/notebooks/ag_2L.gmap

def read_ag_gmap(contig):
    if contig in {"2RL", "3RL"}:
        chrom = contig[0]
        contig_r, contig_l = f"{chrom}R", f"{chrom}L"
        df_r = read_ag_gmap(contig_r)
        df_l = read_ag_gmap(contig_l)
        max_ppos = df_r["pposition"].iloc[-1]
        max_gpos = df_r["gposition"].iloc[-1]
        df_l = df_l.iloc[1:]
        df_l["pposition"] += max_ppos
        df_l["gposition"] += max_gpos
        df = pd.concat([df_r, df_l], axis=0, ignore_index=True)
    else:
        df = pd.read_csv(f"#https://raw.githubusercontent.com/anopheles-genomic-surveillance/selection-atlas/main/workflow/notebooks/ag_{contig}.gmap", sep="\t")
    return df

def ag_gmap(contig):
    ag3 = malariagen_data.Ag3()
        
    # read in the genetic map file
    df_gmap = read_ag_gmap(contig)

    # set up an array of per-base recombination rate values
    rr = np.zeros(len(ag3.genome_sequence(contig)), dtype="f8")

    # fill in the recombination rate values from the genetic map file
    for row, next_row in zip(
        itertools.islice(df_gmap.itertuples(), 0, len(df_gmap)-1), 
        itertools.islice(df_gmap.itertuples(), 1, None)):
        
        # N.B., the genetic map file is in units of cM / Mbp
        # we multiple by 1e-6 to convert to cM / bp
        rr[row.pposition-1:next_row.pposition] = row.rrate * 1e-6
    
    # compute mapping from physical to genetic position
    gmap = np.cumsum(rr)
        
    return gmap

def ag_p2g(contig, ppos):
    """Convert physical position (bp) to genetic position (cM)."""
    gmap = ag_gmap(contig)
    gpos = gmap[ppos - 1]
    return gpos

def ag_g2p(contig, gpos):
    gmap = ag_gmap(contig)
    ppos = bisect_left(gmap, gpos) + 1
    return ppos

In [9]:
def detect_and_plot_ibd_segments(
    self,
    sample_sets,
    sample_query,
    cohort_id,
    site_mask,
    min_ibd1_snps,
    out_dir="../../results/ibd/",
    contigs=('2RL', '3RL', 'X'),
):
    # Detect IBD1 segments
    for contig in contigs:
        
        detect_ibd_segments(
            self=self,
            contig=contig, 
            sample_sets=sample_sets, 
            sample_query=sample_query,
            cohort_id=cohort_id,
            out_dir=out_dir,
            site_mask=site_mask, 
        )
    
    # Load IBD zarr data
    ibd_df = load_ibd_zarr_to_dataframe(
        self=self, 
        contigs=contigs,
        cohort_id=cohort_id,
        out_dir=out_dir, 
    )
    
    # Calculate genome fraction in IBD1
    ibd_fractions_df = fraction_genome_ibd(
        self=self, 
        ibd_df=ibd_df,  
        min_ibd1_snps=min_ibd1_snps, 
        out_dir=out_dir, 
        cohort_id=cohort_id
    )
     
    # Plot IBD segments
    plot_ibd_segments(
        self=self,
        df=ibd_df,
        min_ibd1_snps=min_ibd1_snps,
        show=False,
        out_dir=out_dir,
        cohort_id=cohort_id
        )
    
    return ibd_fractions_df

In [10]:
ag3 = malariagen_data.Ag3(
    results_cache="../../malariagen_data_cache", 
    simple_cache=dict(cache_storage="../../gcs_cache")
)


# af1 = malariagen_data.Af1(
#     pre=False, 
#     results_cache="malariagen_data_cache", 
#     simple_cache=dict(cache_storage="gcs_cache"))

# additional_metadata = pd.read_csv("../../workflow/scripts/additional_sample_metadata.csv")
# af1.add_extra_metadata(additional_metadata)

In [15]:
sample_set = "1244-VO-GH-YAWSON-VMF00149"
df_samples = ag3.sample_metadata(sample_sets=sample_set)

#### Pre-processing

The functions are much much quicker (10x ish) if we remove invariant sites and singletons and doubletons. The size of the array reduces from around 80million SNPs to about 5million by doing this. 

We dont need singletons, as by definition they cannot break a scan which looks for positions which are homozygous but different. Similarly, doubletons could only break it if the both alleles of the doubleton are found in the same individual mosquito - very unlikely. 

### Run algorithm on each contig
Which will write to a .tsv file (large cohorts use too much memory to store as a pd.DataFrame. 

In [16]:
coh_dict = {}
for tax in ['gambiae', 'coluzzii']:
    for loc in df_samples.location.unique():
        loc = loc.split(".")[0]
        query = f"location.str.contains('{loc}') and taxon == '{tax}'"
        df = df_samples.query(query, engine='python')
        if df.shape[0] > 0:
            coh_dict[f"{tax}-{loc}"] = query

In [ ]:
ibd_fraction_df = detect_and_plot_ibd_segments(
        self=ag3,
        sample_sets=sample_set, 
        sample_query=None, #f"cohort_admin2_year == '{coh}'",
        cohort_id="obuasi",
        min_ibd1_snps=2000,
        site_mask='gamb_colu_arab', 
        out_dir='results/obuasi_ibd/',
    )

Loading genotypes


In [ ]:
# self=af1
# contigs = ('2RL', '3RL', 'X')
# cohort_id = df_samples.geographic_cohort[0]
# out_dir= "../../results/ibd/"
# min_ibd1_snps = 10000

# # Load IBD zarr data
# ibd_df = load_ibd_zarr_to_dataframe(
#     self=self, 
#     contigs=contigs,
#     cohort_id=cohort_id,
#     out_dir=out_dir, 
# )

# # Calculate genome fraction in IBD1
# ibd_fractions_df = fraction_genome_ibd(
#     self=self, 
#     ibd_df=ibd_df,  
#     min_ibd1_snps=min_ibd1_snps, 
#     out_dir=out_dir, 
#     cohort_id=cohort_id
# )

# # Plot IBD segments
# plot_ibd_segments(
#     self=self,
#     df=ibd_df,
#     min_ibd1_snps=min_ibd1_snps,
#     out_dir=None,
#     show=True,
#     cohort_id=cohort_id
#     )


In [ ]:
# # Plot IBD segments
# fig = plot_ibd_segments(
#     self=self,
#     df=ibd_df,
#     filter_expr=pl.col("n_snps") > 10_000,
#     show=True,
#     out_dir=None,
#     cohort_id=cohort_id
# )

In [ ]:
# # Plot IBD segments
# plot_ibd_segments(
#     self=self,
#     df=ibd_df,
#     filter_expr=pl.col("n_snps") > 10_000,
#     show=True,
#     out_dir=None,
#     cohort_id=cohort_id
#     )


### Bokeh plots

In [ ]:
# plot_ibd_segments(
#     ag3,
#     ibd1_df,
#     min_ibd1_snps=1000,
#     sort_by='size'
# )